# Hack to fix errors in the SQL Alchemy python library  

> A very lightweight script to be run on `sqla` code that has been freshly generated from LinkML. This adds polymorphism to children of InformationContentEntity classes and adds primaryjoin/secondaryjoin to self-referential relationships.

In [ ]:
#| default_exp hacks.sqla_fixes

In [1]:
#| export

import os
import re
import local_resources.linkml as linkml
from importlib_resources import files

In [3]:
#| export

sqla_source_code = files(linkml).joinpath('schema_sqla.py').read_text()

source = '''has_notes = relationship\( "Note", secondary="Note_has_notes"\)'''
target = '''has_notes = relationship( "Note", 
                             secondary="Note_has_notes", 
                             primaryjoin="Note.id==Note_has_notes.c.Note_id",
                             secondaryjoin="Note.id==Note_has_notes.c.has_notes_id",
                             back_populates="is_about")'''
if re.search(source, sqla_source_code):
    sqla_source_code = re.sub(source, target, sqla_source_code)



In [4]:
#| export

source = '''(return f"InformationContentEntity\\(creation_date={.*\n+\s+#.*\n\s+__mapper_args__ = )(\{\n.*?\n\s+\})\n'''
target = '''\g<1>{
        'concrete': True,
        'polymorphic_on': type,
        "polymorphic_identity": "InformationContentEntity",
    }'''
if re.search(source, sqla_source_code, re.MULTILINE):
    sqla_source_code = re.sub(source, target, sqla_source_code, re.MULTILINE)


In [7]:
#| export

source = '''(return f"<<<SUBHERE>>>\\(.*\n+\s+#.*\n\s+__mapper_args__ = )(\{\n.*?\n\s+\})\n'''
target = '''\g<1>{
        'concrete': True,
        "polymorphic_identity": "<<<SUBHERE>>>",
    }'''
classes_to_edit = ['Note', 
                   'ScientificKnowledgeCollection', 
                   'ScientificKnowledgeExpression', 
                   'ScientificKnowledgeFragment',
                   'ScientificKnowledgeItem']
for c in classes_to_edit:
    s = source.replace('<<<SUBHERE>>>', c)
    t = target.replace('<<<SUBHERE>>>', c)
    if re.search(s, sqla_source_code, re.MULTILINE):
        sqla_source_code = re.sub(s, t, sqla_source_code, re.MULTILINE)

In [8]:
#| export

source = '''has_notes = relationship\( "Note", secondary="<<<SUBHERE>>>_has_notes"\)\n'''
target = '''has_notes = relationship( "Note", secondary="<<<SUBHERE>>>_has_notes", back_populates="is_about")\n'''
classes_to_edit = ['InformationContentEntity',
                   'Note', 
                   'ScientificKnowledgeCollection', 
                   'ScientificKnowledgeExpression', 
                   'ScientificKnowledgeFragment',
                   'ScientificKnowledgeItem']
for c in classes_to_edit:
    s = source.replace('<<<SUBHERE>>>', c)
    t = target.replace('<<<SUBHERE>>>', c)
    if re.search(s, sqla_source_code, re.MULTILINE):
        sqla_source_code = re.sub(s, t, sqla_source_code, re.MULTILINE)

In [9]:
#| export

source = '''is_about = relationship\( "InformationContentEntity", secondary="Note_is_about"\)'''
target = '''is_about = relationship( "InformationContentEntity", secondary="Note_is_about", back_populates="has_notes")'''
if re.search(source, sqla_source_code, re.MULTILINE):
    sqla_source_code = re.sub(source, target, sqla_source_code, re.MULTILINE)

In [10]:
#| export

os.rename(files(linkml).joinpath('schema_sqla.py'), files(linkml).joinpath('schema_sqla_original.py'))
files(linkml).joinpath('schema_sqla.py').write_text(sqla_source_code)

39684